In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import beamsim
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import current_thread
import os
import time

# Set seaborn style for beautiful plots
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

# Default configuration parameters based on beamsim-b0ee7a4.yaml
DEFAULT_CONFIG = {
    'backend': 'ns3-direct',
    'snark1_pull': True,
    'snark1_half_direct': True,
    'shuffle': False,
    'random_seed': 42,
    'group_count': 8,
    'group_validator_count': 512,
    'group_local_aggregator_count': 51,
    'global_aggregator_count': 51,
    'mesh_n': 8,
    'non_mesh_n': 4,
    'signature_time': '20ms',
    'signature_size': 3072,
    'snark_size': 131072,
    'snark1_threshold': 0.75,
    'snark2_threshold': 0.66,
    'aggregation_rate_per_sec': 1000,
    'snark_recursion_aggregation_rate_per_sec': 10,
    'pq_signature_verification_time': '30us',
    'snark_proof_verification_time': '2ms',
    'gml': 'shadow-atlas.bin'
}

In [8]:
def generate_yaml_config(topology="gossip", **overrides):
    """
    Generate a YAML configuration with default values and optional overrides.
    
    Args:
        topology: Network topology ("gossip", "grid", etc.)
        **overrides: Dictionary of parameter overrides
        
    Common override parameters:
        - signature_size: Size of signatures in bytes
        - mesh_n: Gossip mesh_n parameter
        - non_mesh_n: Gossip non_mesh_n parameter
        - group_validator_count: Number of validators per group
        - group_local_aggregator_count: Number of local aggregators per group
        - group_count: Number of groups
        - global_aggregator_count: Number of global aggregators
        - snark1_threshold: Threshold for SNARK1 generation
        - aggregation_rate_per_sec: Rate of signature aggregation
        - random_seed: Random seed for reproducibility
    """
    
    # Use default configuration and add topology, then apply overrides
    defaults = DEFAULT_CONFIG.copy()
    defaults['topology'] = topology
    
    # Apply overrides
    config = {**defaults, **overrides}
    
    # Build the YAML string
    yaml_content = f"""
# Simulation Backend Configuration
backend: {config['backend']}
                                
snark1_pull: {str(config['snark1_pull']).lower()}
snark1_half_direct: {str(config['snark1_half_direct']).lower()}

# Network Topology Configuration
topology: {config['topology']}

# Whether to shuffle validators from the same group to different routers
shuffle: {str(config['shuffle']).lower()}

# Seed for reproducible simulation results
random_seed: {config['random_seed']}

# Role Assignment Configuration
roles:
  group_count: {config['group_count']}
  group_validator_count: {config['group_validator_count']}
  group_local_aggregator_count: {config['group_local_aggregator_count']}
  global_aggregator_count: {config['global_aggregator_count']}

# Gossipsub Network Configuration
gossip:
  mesh_n: {config['mesh_n']}
  non_mesh_n: {config['non_mesh_n']}

# Cryptographic Constants
consts:
  signature_time: {config['signature_time']}
  signature_size: {config['signature_size']}
  snark_size: {config['snark_size']}
  snark1_threshold: {config['snark1_threshold']}
  snark2_threshold: {config['snark2_threshold']}
  aggregation_rate_per_sec: {config['aggregation_rate_per_sec']}
  snark_recursion_aggregation_rate_per_sec: {config['snark_recursion_aggregation_rate_per_sec']}
  pq_signature_verification_time: {config['pq_signature_verification_time']}
  snark_proof_verification_time: {config['snark_proof_verification_time']}

# Network Simulation Parameters
network:
  gml: "{config['gml']}"
"""
    
    return beamsim.yaml(yaml_content.strip())

In [9]:
# Worker function for parallel processing
def _run_single_simulation_thread(topology, param_value, parameter_name, param_display, 
                                 config_overrides, run_kwargs, display_name, display_unit):
    """
    Worker function to run a single simulation in parallel using threading.
    
    Args:
        Individual parameters instead of tuple to avoid pickle issues
    
    Returns:
        Dictionary with simulation results
    """
    try:
        # Create config with this parameter value
        current_config = {**config_overrides, parameter_name: param_value}
        modified_yaml = generate_yaml_config(topology=topology, **current_config)
        
        # Run simulation with modified config (global aggregation mode)
        current_run_kwargs = {**run_kwargs, 'c': modified_yaml}
        items = beamsim.run(**current_run_kwargs, t=topology)
        
        # Extract SNARK2 timing information for global aggregation analysis
        snark2_sent = beamsim.filter_report(items, "snark2_sent")
        if snark2_sent and len(snark2_sent) > 0:
            snark2_completion_time = snark2_sent[0][0] if snark2_sent[0] else 0
            
            # Also get SNARK1 timing for reference
            snark1_sent = beamsim.get_snark1_sent(items)
            snark1_completion_time = snark1_sent[0][-1] if snark1_sent and len(snark1_sent[0]) > 0 else 0
            
            # Get simulation info
            _, _, validator_count, snark1_threshold, snark2_threshold = beamsim.filter_report(items, "info")[0]
            
            return {
                'topology': topology,
                'topology_display': beamsim.topology_name[topology],
                'parameter_name': parameter_name,
                'parameter_value': param_value,
                'parameter_display': param_display,
                'snark1_completion_time': snark1_completion_time,
                'snark2_completion_time': snark2_completion_time,
                'validator_count': validator_count,
                'snark1_threshold': snark1_threshold,
                'snark2_threshold': snark2_threshold,
                'success': True
            }
        else:
            return {
                'topology': topology,
                'parameter_name': parameter_name,
                'parameter_value': param_value,
                'parameter_display': param_display,
                'success': False,
                'error': 'No SNARK2 data found'
            }
    
    except Exception as e:
        return {
            'topology': topology,
            'parameter_name': parameter_name,
            'parameter_value': param_value,
            'parameter_display': param_display,
            'success': False,
            'error': str(e)
        }


def theoretical_minimum_snark2_time(group_count, snark1_threshold, snark_size, snark_agg_rate):
    """
    Calculate the theoretical minimum SNARK2 aggregation time based on parameters.
    
    Args:
        group_count: Number of groups
        snark1_threshold: Threshold for SNARK1 generation (0-1)
        snark_size: Size of each SNARK proof in bytes
        snark_agg_rate: Rate of SNARK aggregation per second
        
    Returns:
        Theoretical minimum SNARK2 aggregation time in milliseconds
    """
    # Calculate total number of SNARK1 proofs to aggregate
    snark1_proofs_to_aggregate = group_count  # One SNARK1 proof per group
    
    # Calculate time to aggregate at the given rate
    agg_time_sec = snark1_proofs_to_aggregate / snark_agg_rate  # in seconds
    
    return agg_time_sec * 1000  # Convert to milliseconds

In [10]:
def run_parameter_analysis(parameter_name, parameter_values, topologies=["gossip", "grid"], 
                          display_name=None, display_unit="", parallel=True, max_workers=None, 
                          **base_config):
    """
    Generic function to analyze the effect of changing a parameter on global aggregation time.
    
    Args:
        parameter_name: Name of the parameter to vary (e.g., 'group_count', 'global_aggregator_count')
        parameter_values: List of values to test for the parameter
        topologies: List of network topologies to test
        display_name: Human-readable name for the parameter (defaults to parameter_name)
        display_unit: Unit to display after the parameter value (e.g., "groups", "aggregators")
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: min(8, cpu_count()))
        **base_config: Base configuration parameters and run kwargs
        
    Returns:
        DataFrame with results, with parameter order preserved
    """
    
    if display_name is None:
        display_name = parameter_name.replace('_', ' ').title()
    
    # Separate run kwargs from config overrides
    run_kwargs_keys = {'mpi', 'c', 'b', 'g', 'gv', 'la', 'ga', 'shuffle', 't'}
    run_kwargs = {k: v for k, v in base_config.items() if k in run_kwargs_keys}
    config_overrides = {k: v for k, v in base_config.items() if k not in run_kwargs_keys}
    
    # Determine the number of workers
    if max_workers is None:
        max_workers = min(8, len(parameter_values) * len(topologies))
    
    print(f"Testing {display_name}: {parameter_values}")
    print(f"Topologies: {topologies}")
    
    if parallel:
        print(f"Running simulations in parallel with {max_workers} workers (ThreadPoolExecutor)...\n")
    else:
        print("Running simulations sequentially...\n")
    
    # Prepare all simulation parameters
    simulation_params = []
    for topology in topologies:
        for param_value in parameter_values:
            param_display = f"{param_value} {display_unit}".strip() if param_value is not None else "Default"
            simulation_params.append((
                topology, param_value, parameter_name, param_display,
                config_overrides, run_kwargs, display_name, display_unit
            ))
    
    results = []
    
    if parallel and len(simulation_params) > 1:
        # Run simulations in parallel using ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            future_to_params = {}
            for params in simulation_params:
                topology, param_value, parameter_name, param_display, config_overrides, run_kwargs, display_name, display_unit = params
                future = executor.submit(_run_single_simulation_thread, 
                                       topology, param_value, parameter_name, param_display,
                                       config_overrides, run_kwargs, display_name, display_unit)
                future_to_params[future] = params
            
            # Process completed tasks
            total_sims = len(simulation_params)
            completed = 0
            
            for future in as_completed(future_to_params):
                completed += 1
                result = future.result()
                
                if result['success']:
                    results.append(result)
                    topology_display = beamsim.topology_name.get(result['topology'], result['topology'])
                    print(f"  [{completed}/{total_sims}] {topology_display} - {result['parameter_display']}: "
                          f"SNARK1: {result['snark1_completion_time']:.1f} ms, "
                          f"SNARK2: {result['snark2_completion_time']:.1f} ms")
                else:
                    print(f"  [{completed}/{total_sims}] ERROR - {result['topology']} - {result['parameter_display']}: "
                          f"{result['error']}")
    else:
        # Run simulations sequentially
        for i, params in enumerate(simulation_params):
            topology, param_value, parameter_name, param_display, config_overrides, run_kwargs, display_name, display_unit = params
            print(f"  [{i+1}/{len(simulation_params)}] Testing {topology} - {param_display}")
            
            result = _run_single_simulation_thread(
                topology, param_value, parameter_name, param_display,
                config_overrides, run_kwargs, display_name, display_unit
            )
            
            if result['success']:
                results.append(result)
                print(f"    → SNARK1: {result['snark1_completion_time']:.1f} ms, SNARK2: {result['snark2_completion_time']:.1f} ms")                
            else:
                print(f"    → ERROR: {result['error']}")
    
    # Convert to DataFrame and preserve parameter order
    df = pd.DataFrame(results)
    if not df.empty:
        # Add an order column to preserve the original parameter order
        param_to_order = {param: i for i, param in enumerate(parameter_values)}
        df['parameter_order'] = df['parameter_value'].map(param_to_order)
        # Sort by topology and parameter order to ensure consistent ordering
        df = df.sort_values(['topology', 'parameter_order']).reset_index(drop=True)
    
    return df


def plot_parameter_analysis(results_df, parameter_name, display_name=None, display_unit="", 
                          title_suffix="", figure_size=(14, 8), include_theoretical=True,
                          **base_config):
    """
    Plot the results of a parameter analysis for global aggregation.
    
    Args:
        results_df: DataFrame from run_parameter_analysis
        parameter_name: Name of the parameter that was varied
        display_name: Human-readable name for the parameter
        display_unit: Unit for the parameter
        title_suffix: Additional text for the plot title
        figure_size: Size of the plot figure
        include_theoretical: Whether to include theoretical minimum line
        **base_config: Base configuration parameters for theoretical calculations
    """
    
    if display_name is None:
        display_name = parameter_name.replace('_', ' ').title()
    
    # Create the plot
    plt.figure(figsize=figure_size)
    
    # Plot lines for each topology
    topologies = results_df['topology'].unique()
    colors = sns.color_palette("husl", len(topologies))
    
    # Get parameter order from the dataframe (preserves original input order)
    if 'parameter_order' in results_df.columns:
        ordered_params = results_df.sort_values('parameter_order')['parameter_display'].unique()
    else:
        ordered_params = results_df['parameter_display'].unique()
    
    for i, topology in enumerate(topologies):
        topo_data = results_df[results_df['topology'] == topology].copy()
        
        # Sort by parameter order to maintain input sequence
        if 'parameter_order' in topo_data.columns:
            topo_data = topo_data.sort_values('parameter_order')
        else:
            try:
                topo_data = topo_data.sort_values('parameter_value')
            except:
                pass
        
        x_values = list(range(len(topo_data)))
        
        # Plot both SNARK1 and SNARK2 times
        y_values_snark1 = topo_data['snark1_completion_time'].tolist()
        y_values_snark2 = topo_data['snark2_completion_time'].tolist()
        
        plt.plot(
            x_values,
            y_values_snark1,
            marker='o',
            linewidth=2,
            markersize=6,
            color=colors[i],
            linestyle='--',
            label=f'{topo_data.iloc[0]["topology_display"]} - SNARK1',
            alpha=0.7
        )
        
        plt.plot(
            x_values,
            y_values_snark2,
            marker='s',
            linewidth=3,
            markersize=8,
            color=colors[i],
            label=f'{topo_data.iloc[0]["topology_display"]} - SNARK2 (Global)',
            alpha=0.8
        )
    
    # Add theoretical minimum line if requested
    if include_theoretical and len(results_df) > 0:
        theoretical_params = {**DEFAULT_CONFIG, **base_config}
        
        theoretical_times = []
        sample_data = results_df.sort_values('parameter_order' if 'parameter_order' in results_df.columns else 'parameter_value')
        unique_param_values = sample_data['parameter_value'].unique()
        
        for param_value in unique_param_values:
            current_params = theoretical_params.copy()
            current_params[parameter_name] = param_value
            
            theoretical_time = theoretical_minimum_snark2_time(
                group_count=current_params['group_count'],
                snark1_threshold=current_params['snark1_threshold'],
                snark_size=current_params['snark_size'],
                snark_agg_rate=current_params['snark_recursion_aggregation_rate_per_sec']
            )
            theoretical_times.append(theoretical_time)
        
        # Plot theoretical minimum line
        x_values = list(range(len(theoretical_times)))
        plt.plot(
            x_values,
            theoretical_times,
            linestyle=':',
            linewidth=2,
            color='red',
            label='Theoretical Minimum (SNARK2)',
            alpha=0.8
        )
    
    # Set axis labels and title
    plt.xlabel(f'{display_name} {display_unit}'.strip(), fontweight='bold', fontsize=12)
    plt.ylabel('Aggregation Time (ms)', fontweight='bold', fontsize=12)
    
    title = f'Effect of {display_name} on Global Aggregation Time'
    if title_suffix:
        title += f'\n{title_suffix}'
    title += '\n(Global Aggregation Analysis)'
    plt.title(title, fontweight='bold', fontsize=14)
    
    # Set x-axis tick labels using preserved parameter order
    plt.xticks(range(len(ordered_params)), ordered_params, rotation=45)
    
    plt.legend(frameon=True, fancybox=True, shadow=True, fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # Add performance annotations
    if len(results_df) > 0:
        min_time = results_df['snark2_completion_time'].min()
        max_time = results_df['snark2_completion_time'].max()
        time_diff = max_time - min_time
        improvement = (time_diff / max_time) * 100
        
        plt.text(0.02, 0.98, 
                f'SNARK2 time difference: {time_diff:.1f}ms\n'
                f'Performance improvement: {improvement:.1f}%',
                transform=plt.gca().transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
                fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print(f"{display_name.upper()} EFFECT SUMMARY - GLOBAL AGGREGATION")
    print("="*60)
    
    if include_theoretical and len(results_df) > 0:
        print(f"\nTheoretical Minimum (SNARK2):")
        print(f"  Time range: {min(theoretical_times):.1f} - {max(theoretical_times):.1f} ms")
    
    for topology in topologies:
        topo_data = results_df[results_df['topology'] == topology]
        if len(topo_data) > 0:
            print(f"\n{topo_data.iloc[0]['topology_display']} Topology:")
            print(f"  SNARK1 time range: {topo_data['snark1_completion_time'].min():.1f} - {topo_data['snark1_completion_time'].max():.1f} ms")
            print(f"  SNARK2 time range: {topo_data['snark2_completion_time'].min():.1f} - {topo_data['snark2_completion_time'].max():.1f} ms")
            
            for _, row in topo_data.iterrows():
                print(f"    {row['parameter_display']:>12}: SNARK1: {row['snark1_completion_time']:.1f} ms, SNARK2: {row['snark2_completion_time']:.1f} ms")


def analyze_parameter_effect(parameter_name, parameter_values, topologies=["gossip", "grid"], 
                           display_name=None, display_unit="", title_suffix="", 
                           parallel=True, max_workers=None, **base_config):
    """
    Complete analysis workflow: run simulations and plot results for global aggregation.
    
    Args:
        parameter_name: Name of the parameter to vary
        parameter_values: List of values to test
        topologies: List of topologies to test
        display_name: Human-readable parameter name
        display_unit: Unit for the parameter
        title_suffix: Additional text for plot title
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers
        **base_config: Base configuration and run parameters
        
    Returns:
        DataFrame with results
    """
    
    # Run the analysis
    results_df = run_parameter_analysis(
        parameter_name, parameter_values, topologies, 
        display_name, display_unit, parallel=parallel, 
        max_workers=max_workers, **base_config
    )
    
    # Plot the results
    plot_parameter_analysis(
        results_df, parameter_name, display_name, 
        display_unit, title_suffix, **base_config
    )
    
    return results_df

In [11]:
def analyze_group_count_effect(group_counts, topologies=["gossip", "grid"], 
                              parallel=True, max_workers=None, **base_config):
    """
    Analyze how the number of groups affects global aggregation time.
    
    Args:
        group_counts: List of group counts to test
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='group_count',
        parameter_values=group_counts,
        topologies=topologies,
        display_name='Number of Groups',
        display_unit='groups',
        title_suffix='Global Aggregation Scalability',
        parallel=parallel,
        max_workers=max_workers,
        **base_config
    )


def analyze_global_aggregator_count_effect(global_aggregator_counts, topologies=["gossip", "grid"], 
                                         parallel=True, max_workers=None, **base_config):
    """
    Analyze how the number of global aggregators affects global aggregation time.
    
    Args:
        global_aggregator_counts: List of global aggregator counts to test
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='global_aggregator_count',
        parameter_values=global_aggregator_counts,
        topologies=topologies,
        display_name='Number of Global Aggregators',
        display_unit='aggregators',
        title_suffix='Global Aggregation Capacity',
        parallel=parallel,
        max_workers=max_workers,
        **base_config)

In [12]:
# Base configuration parameters - using beamsim-b0ee7a4.yaml defaults
base_config = {
    'mpi': 9,  # Set to False to disable MPI, or number of processes to enable
}

# Test different numbers of groups for global aggregation analysis
group_counts = [6, 8, 10, 12, 14, 16]

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of number of groups on global aggregation time...")
print("Using parameters from beamsim-b0ee7a4.yaml as defaults.\n")

# Run the group count effect analysis
group_results_df = analyze_group_count_effect(group_counts, topologies, max_workers=6, **base_config, parallel=False)

# Analyze the effect of number of global aggregators on global aggregation time
global_aggregator_counts = [10, 30, 50, 70, 90]

# Base configuration parameters
base_config = {
    'mpi': 9,
}

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of number of global aggregators on global aggregation time...")
print("Using parameters from beamsim-b0ee7a4.yaml as defaults.\n")

# Run the global aggregator count effect analysis
global_agg_results_df = analyze_global_aggregator_count_effect(global_aggregator_counts, topologies, max_workers=5, **base_config)


# Combined analysis: Compare the effects of both parameters
print("="*80)
print("COMBINED ANALYSIS SUMMARY")
print("="*80)

print("\nGroup Count Analysis Summary:")
if not group_results_df.empty:
    for topology in group_results_df['topology'].unique():
        topo_data = group_results_df[group_results_df['topology'] == topology]
        topology_name = topo_data.iloc[0]['topology_display']
        min_snark2 = topo_data['snark2_completion_time'].min()
        max_snark2 = topo_data['snark2_completion_time'].max()
        print(f"  {topology_name}: SNARK2 range {min_snark2:.1f} - {max_snark2:.1f} ms (Δ{max_snark2-min_snark2:.1f}ms)")

print("\nGlobal Aggregator Count Analysis Summary:")
if not global_agg_results_df.empty:
    for topology in global_agg_results_df['topology'].unique():
        topo_data = global_agg_results_df[global_agg_results_df['topology'] == topology]
        topology_name = topo_data.iloc[0]['topology_display']
        min_snark2 = topo_data['snark2_completion_time'].min()
        max_snark2 = topo_data['snark2_completion_time'].max()
        print(f"  {topology_name}: SNARK2 range {min_snark2:.1f} - {max_snark2:.1f} ms (Δ{max_snark2-min_snark2:.1f}ms)")

# Create a combined comparison plot
if not group_results_df.empty and not global_agg_results_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Plot group count effects
    topologies = group_results_df['topology'].unique()
    colors = sns.color_palette("husl", len(topologies))
    
    for i, topology in enumerate(topologies):
        topo_data = group_results_df[group_results_df['topology'] == topology].sort_values('parameter_order')
        x_values = list(range(len(topo_data)))
        y_values = topo_data['snark2_completion_time'].tolist()
        
        ax1.plot(x_values, y_values, marker='o', linewidth=3, markersize=8, 
                color=colors[i], label=f'{topo_data.iloc[0]["topology_display"]}', alpha=0.8)
    
    ax1.set_xlabel('Number of Groups', fontweight='bold', fontsize=12)
    ax1.set_ylabel('SNARK2 Completion Time (ms)', fontweight='bold', fontsize=12)
    ax1.set_title('Effect of Group Count on Global Aggregation', fontweight='bold', fontsize=14)
    ordered_group_params = group_results_df.sort_values('parameter_order')['parameter_display'].unique()
    ax1.set_xticks(range(len(ordered_group_params)))
    ax1.set_xticklabels(ordered_group_params, rotation=45)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot global aggregator count effects
    for i, topology in enumerate(topologies):
        topo_data = global_agg_results_df[global_agg_results_df['topology'] == topology].sort_values('parameter_order')
        x_values = list(range(len(topo_data)))
        y_values = topo_data['snark2_completion_time'].tolist()
        
        ax2.plot(x_values, y_values, marker='s', linewidth=3, markersize=8, 
                color=colors[i], label=f'{topo_data.iloc[0]["topology_display"]}', alpha=0.8)
    
    ax2.set_xlabel('Number of Global Aggregators', fontweight='bold', fontsize=12)
    ax2.set_ylabel('SNARK2 Completion Time (ms)', fontweight='bold', fontsize=12)
    ax2.set_title('Effect of Global Aggregator Count on Global Aggregation', fontweight='bold', fontsize=14)
    ordered_agg_params = global_agg_results_df.sort_values('parameter_order')['parameter_display'].unique()
    ax2.set_xticks(range(len(ordered_agg_params)))
    ax2.set_xticklabels(ordered_agg_params, rotation=45)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle('Global Aggregation Analysis: Group Count vs Global Aggregator Count', 
                fontweight='bold', fontsize=16, y=0.98)
    plt.tight_layout()
    plt.show()

print("\n" + "="*80)
print("KEY INSIGHTS:")
print("="*80)
print("1. Group Count: More groups = more SNARK1 proofs to aggregate globally")
print("2. Global Aggregators: More aggregators = better parallelization of global aggregation")
print("3. Compare the relative impact of each parameter on SNARK2 completion time")
print("4. Grid topology typically shows different scaling characteristics than Gossip")

Analyzing the effect of number of groups on global aggregation time...
Using parameters from beamsim-b0ee7a4.yaml as defaults.

Testing Number of Groups: [6, 8, 10, 12, 14, 16]
Topologies: ['gossip', 'grid']
Running simulations sequentially...

  [1/12] Testing gossip - 6 groups
run: mpirun -n 9 build/beamsim -c /var/folders/r9/53ggp_3x6w51pqj8_kv1_lb00000gn/T/beamsim-yaml-md5/c836619fb583756fb6f66b52c204ed0f -t gossip --report
    → SNARK1: 1134.0 ms, SNARK2: 2568.0 ms
  [2/12] Testing gossip - 8 groups
run: mpirun -n 9 build/beamsim -c /var/folders/r9/53ggp_3x6w51pqj8_kv1_lb00000gn/T/beamsim-yaml-md5/dee91170c8e4e2a728df485ac57d09b5 -t gossip --report
    → SNARK1: 1109.0 ms, SNARK2: 2755.0 ms
  [3/12] Testing gossip - 10 groups
run: mpirun -n 9 build/beamsim -c /var/folders/r9/53ggp_3x6w51pqj8_kv1_lb00000gn/T/beamsim-yaml-md5/dc14de6ed053ba5b4c8f217d708cdf69 -t gossip --report
    → SNARK1: 1207.0 ms, SNARK2: 2762.0 ms
  [4/12] Testing gossip - 12 groups
run: mpirun -n 9 build/beams

KeyboardInterrupt: 